In [1]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/wk247/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [2]:
import transformers

print(transformers.__version__)

4.12.5


### Fine-tuning a model on a text classification task

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [4]:
task = "mnli"
model_checkpoint = "typeform/distilbert-base-uncased-mnli"
batch_size = 16

### load dataset

In [5]:
from datasets import load_dataset, load_metric

In [7]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /home/wk247/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [10]:
dataset["train"][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(dataset["train"])

,premise,hypothesis,label,idx
0,oh well we do have a Kmart here you better believe it isn't there a Kmart everywhere,I like to shop at Kmart for stuff I cannot find at the local shops.,neutral,148357
1,i don't i don't miss the cold at all but i don't mind the heat,I hate the heat and wish I still lived somewhere cold.,contradiction,202815
2,oh you don't,You will in the future.,neutral,201077
3,The excellent Youth Wing of the museum for children and teenagers is opposite.,The museum is strictly for adults only.,contradiction,207266
4,"I had an idea, but clearly I was mistaken.",The idea probably wasn't correct.,neutral,190988
5,"At the Yale Law School Lowenstein International Human Rights Clinic, she codirected a litigation effort on behalf of Haitian refugees in federal challenge to the United States interdiction program, which included testifying before the U.S.",Testifying for the US is crucial for determining a group's validity.,neutral,105487
6,"Marcel Proust spent many summers at its splendid Grand H??tel, where he wrote part of his A la Recherche du Temps Perdu.",Marcel Proust is the author of A la Recherche du Temps Perdu.,entailment,324734
7,This proved highly dangerous to pilgrims who approached the abbey across the sands (the causeway joining the island to the mainland was not built until 1874).,The church across the sands proved highly dangerous to travelers and the bridge from the island to the mainland was not built until 1874.,entailment,125280
8,i had to do something to just elevate myself a little bit off their level i mean you want to be friends with the students but you don't quite want to be their equal,"You can be friendly with the students, but must be above their level.",entailment,246635
9,"Haifa's most famous attraction is the splendid golden-domed Baha'i Shrine and Gardens (entrance on Zionism Avenue; Shrine open daily 9:00 a.m. to noon, gardens open daily 8:00 a.m to 5:00 p.m. ), the international headquarters of the 4-million-strong Baha'i faith, which was founded by the Persian holy man, Baha'u'llah (who died in 1892).","Haifa is most famous for its ancient ruins, which encircle the city.",contradiction,57862


In [13]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

Note that `load_metric` has loaded the proper metric associated to your task, which is:

* for CoLA: Matthews Correlation Coefficient
* for MNLI (matched or mismatched): Accuracy
* for MRPC: Accuracy and F1 score
* for QNLI: Accuracy
* for QQP: Accuracy and F1 score
* for RTE: Accuracy
* for SST-2: Accuracy
* for STS-B: Pearson Correlation Coefficient and Spearman's_Rank_Correlation_Coefficient
* for WNLI: Accuracy

### Preprocessing the data

In [14]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/258 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [15]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [17]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Conceptually cream skimming has two basic dimensions - product and geography.
Sentence 2: Product and geography are what make cream skimming work. 


In [18]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [19]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 17158, 2135, 6949, 8301, 25057, 2038, 2048, 3937, 9646, 1011, 4031, 1998, 10505, 1012, 102, 4031, 1998, 10505, 2024, 2054, 2191, 6949, 8301, 25057, 2147, 1012, 102], [101, 2017, 2113, 2076, 1996, 2161, 1998, 1045, 3984, 2012, 2012, 2115, 2504, 7910, 2017, 4558, 2068, 2000, 1996, 2279, 2504, 2065, 2065, 2027, 5630, 2000, 9131, 1996, 1996, 6687, 2136, 1996, 13980, 5630, 2000, 2655, 2000, 9131, 1037, 3124, 2013, 6420, 1037, 2059, 1037, 3313, 1037, 3124, 3632, 2039, 2000, 5672, 2032, 1998, 1037, 2309, 1037, 3124, 3632, 2039, 2000, 5672, 2032, 102, 2017, 4558, 1996, 2477, 2000, 1996, 2206, 2504, 2065, 1996, 2111, 9131, 1012, 102], [101, 2028, 1997, 2256, 2193, 2097, 4287, 2041, 2115, 8128, 3371, 2135, 1012, 102, 1037, 2266, 1997, 2026, 2136, 2097, 15389, 2115, 4449, 2007, 14269, 11718, 1012, 102], [101, 2129, 2079, 2017, 2113, 1029, 2035, 2023, 2003, 2037, 2592, 2153, 1012, 102, 2023, 2592, 7460, 2000, 2068, 1012, 102], [101, 3398, 1045, 2425, 2017, 2054, 2295, 2065, 20

In [20]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/393 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [65]:
encoded_dataset.save_to_disk("data/mnli")
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise'],
        num_rows: 9847
    })
})